In [1]:
import numpy as np
import pandas as pd
import streamlit as st
from src.loaders import get_worksheet, get_worksheet_location
import streamlit as st
import pandas as pd

2023-05-27 18:20:59.854 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager
2023-05-27 18:20:59.857 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager
2023-05-27 18:20:59.859 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager


In [2]:
sheet_url = st.secrets["private_gsheets_url"]

In [3]:
cheki_df = get_worksheet(sheet_url, 0)
person_df = get_worksheet(sheet_url, 1)
venue_df = get_worksheet_location(sheet_url, 3)

2023-05-27 18:21:07.692 
  command:

    streamlit run /home/wotashu/.cache/pypoetry/virtualenvs/chekilytics-307NsMUd-py3.11/lib/python3.11/site-packages/ipykernel_launcher.py [ARGUMENTS]
2023-05-27 18:21:07.694 No runtime found, using MemoryCacheStorageManager
2023-05-27 18:21:09.955 No runtime found, using MemoryCacheStorageManager


In [4]:
person_df

,entity,name1,group1,name2,group2,name3,group3,name4,group4,name5,group5
1,勝月蘭丸@1682,勝月蘭丸,1682,,,,,,,,
2,承和 薫@1682,承和 薫,1682,,,,,,,,
3,月見夜零@1682,月見夜零,1682,,,,,,,,
4,神山おだし@1682,神山おだし,1682,,,,,,,,
5,天野ひろこ@.BPM,天野ひろこ,.BPM,,,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...
492,山川 さくら@おちゃメンタル☆パーティー,山川 さくら,おちゃメンタル☆パーティー,,,,,,,,
493,釈迦力まひな@バブルバビデガム,釈迦力まひな,バブルバビデガム,,,,,,,,
494,POLLIE@Zero Project,POLLIE,Zero Project,,,,,,,,
495,花紀 舞乃@CURTiA,花紀 舞乃,CURTiA,,,,,,,,


In [39]:
persons_list = []
for idx, row in person_df.iterrows():
    person_id = idx
    entities = []
    for i in range(5, 0, -1):
        tmp_dict = {}
        if row[f"name{i}"]:
            tmp_dict["name"] = row[f"name{i}"]
        if row[f"group{i}"]:
            tmp_dict["group"] = row[f"group{i}"]
        if tmp_dict:
            entities.append(tmp_dict)
    persons_list.append({"person_id": idx, "entities": entities, "current": row["entity"]})


In [41]:
person_data = pd.DataFrame(persons_list)